#Analyzing the NYC Subway Dataset

##Background Information

In this project, we look at the NYC Subway data and figure out if more people ride the subway when it is raining versus when it is not raining.
The dataset that is available contains data about turnstiles at different stations as well as weather conditions.

##Statistical Test

###Independent, dependent variables

First, let's see what kind of data is in the dataset by exploring the column names.

In [16]:
import pandas as pd
filename='data/improved-dataset/turnstile_weather_v2.csv'
turnstile_weather = pd.read_csv(filename)
print list(turnstile_weather.columns.values)

['UNIT', 'DATEn', 'TIMEn', 'ENTRIESn', 'EXITSn', 'ENTRIESn_hourly', 'EXITSn_hourly', 'datetime', 'hour', 'day_week', 'weekday', 'station', 'latitude', 'longitude', 'conds', 'fog', 'precipi', 'pressurei', 'rain', 'tempi', 'wspdi', 'meanprecipi', 'meanpressurei', 'meantempi', 'meanwspdi', 'weather_lat', 'weather_lon']


The purpose of this project is to find out if more people use the subway when it is raining. Fortunately, there is a column rain that indicates if it was raining at a certain time and location. According to the documentation it is an indicator (0 or 1). 
How to measure if the subway is used more? Well, the data contains the number of entries at some units where a unit is a data collector of multiple turnstiles. ENTRIESn is the reading of cummulative turnstile entries from the remote unit and ENTRIESn_hourly indicates the difference in ENTRIES from the previous REGULAR reading. It seems like ENTRIESn_hourly is a good measure of how many people will use the subway.

Therefore, the independent variable is the rain. It can either be a 0 (no rain) or 1 (rain)
The dependent variable is the the number of entries since the last reading (ENTRIESn_hourly)

### Normal distribution check

The data that is provided is a sample. With the hypothesis tests we want to explore if we can conclude that the subway is used more when it is raining. 

When comparing means of populations, multiple tests can be used. A t-test or z-test can be used if the data is assumed to be normally distributed. 
However, the data from our sample is not normally distributed if we look at the plot below. Also the Shapiro test indicates this, since the p values are 0.0, so the null hypothesis that the data is from a normal distribution is rejected.

In [17]:
import scipy.stats
rain = turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain'] == 1]
norain = turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain'] == 0]
print "shapiro test norain:", scipy.stats.shapiro(norain) 
print "shapiro test rain:", scipy.stats.shapiro(rain) 

shapiro test norain: (0.5956180691719055, 0.0)
shapiro test rain: (0.5938820838928223, 0.0)


![Histogram of rain vs norain values](plots/histogram_rain_norain.png)

The histogram displays the frequency of the ENTRIESn_hourly values split by rain. As can be seen, the lowest ENTRIESn_hourly have the highest frequency. 
For increasing hourly entries, the frequency drops, this trend holds for rain as well as no rain. When comparing rain vs norain, it is clear that the rain situation has a lot less values.
This is quite logical, cause most of the time it is not raining, so there is less data available.

### Non parametric test and Hypothesis

Because the data is not normally distributed, a t or z test cannot be used, but a non parametric test can be used.
Mann–Whitney´s U test is a nonparametrict test that can be used if we don't assume that the data follows a certain distribution. Because we don´t assume any underlying distribution this test will be used.

Mann-Whitney's U test works as follows: when we draw at random some values from each population, is the chance that  
a higher value is obtained for 1 population is equal to 0.5? Let Xn be some random values from the no rain population and Xr random values from the rain population. Let H0 and Ha respectively be the 0 and alternative hypothesis. The hypotheses can thus be written as:

H0 : P (Xn > Xr) = 0.5

Ha : P (Xn > Xr) != 0.5

The test will be two tailed, since I don't want to make an assumption about whether the chance would be less or greather than 0.5. The p critical value is 0.05, since this is quite common.

### Results from executing the test

Below the results from the test can be found.

In [18]:
import numpy as np
print "length norain df vs rain df:", len(norain), len(rain)
print "Norain mean, median:", np.mean(norain), np.median(norain)
print "Rain mean, median:", np.mean(rain), np.median(rain)
U, p = scipy.stats.mannwhitneyu(rain, norain)
print "U-statistic:", U
print "p-value one sided:", p
print "p-value two sided:", 2 * p

length norain df vs rain df: 33064 9585
Norain mean, median: 1845.53943866 893.0
Rain mean, median: 2028.19603547 939.0
U-statistic: 153635120.5
p-value one sided: 2.74106957124e-06
p-value two sided: 5.48213914249e-06


The mannwhitney u test is per default one-tailed. Because the test we want to perform is two tailed, the p-value should by multiplied by 2. 
As can be seen, the (two sided) p-value is a lot smaller than the p critical value of 0.05. Therefore it is very unlikely that P (Xn > Xr) = 0.5 , so the alternative hypothesis applies: the chance that a random sample from the no rain population is greater that a sample from the rain population is not 0.5. 
Can we conclude that the chance is greater than 0.5 or small than 0.5? Well, by looking at the mean and median values it is clear that these values are higher for rain. Thus on average there are more hourly entries and the 50% value is larger in case of rain. Therefore, we can conclude the chance is less than 0.5. 

##Linear Regression

Machine learning is a branch of artificial intelligence focused on constructing systems that learn from large amounts of data to make predictions.
Linear Regression is one of the most used models in machine learning.

### Regression model
To create a lineair regression model I will use the Ordinary Least Squares (OLS) from Statsmodels. 

### Features 

I have used the following features in the regression model

* rain: because in the statistical test, it is clear that rain matters.
* hour: because there are probably more people taking the subway in peak hours.
* meantempi: because there are probably less travelers when it is very cold or hot.
* weekday: because I think some days are more populair to work than others.
* UNIT (dummy variable): because when I used this, it drastically improved my R2 value.

### Parameters of non-dummy features and R-squared

below the parameters of the non-dummy features and R-squared are shown

In [19]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

def linear_regression(features, values):
    """
    Perform linear regression given a data set with an arbitrary number of features. 
    This can be the same code as in the lesson #3 exercise.
    """ 
    features = sm.add_constant(features)
    model = sm.OLS(values, features)
    results = model.fit()
    intercept = results.params[0]
    params = results.params[1:]    
    return intercept, params
    

def compute_r_squared(data, predictions):
    '''
    Calculates R sqared based on given data and predictions
    '''
    SST = ((data - np.mean(data)) **2).sum()
    SSREG = ((predictions - data) **2).sum()
    r_squared = 1 - SSREG / SST
    
    return r_squared

filename='data/improved-dataset/turnstile_weather_v2.csv'
turnstile_weather = pd.read_csv(filename)
features = turnstile_weather[['rain', 'hour', 'meantempi', 'weekday']]
dummy_units = pd.get_dummies(turnstile_weather['UNIT'], prefix='unit')
features = features.join(dummy_units)
values = turnstile_weather['ENTRIESn_hourly']
intercept, params = linear_regression(features, values)
predictions = intercept + np.dot(features, params)
print "param-rain", params['rain']
print "param-hour", params['hour']
print "param-meantempi", params['meantempi']
print "param-weekday", params['weekday']
print "r-squared:", compute_r_squared(values, predictions)

param-rain -38.034135646
param-hour 123.403811602
param-meantempi -13.8580644847
param-weekday 982.860016244
r-squared: 0.482340215273


The R2 value is about 0.48, so what does this mean? R2 is a measure of how well the data fits the regression model and it is also defined by the percentage of the response variable variation that is explained by a linear model. The value of R2 is 1 for a perfect fit and 0 when there is no fit. So and R2 value of 0.48 means that almost 50% of the variation in ridership is explained by the model.

But, R2 doesn't say everything because a high value is not always indicating that there is a good model fit. This can be the case when the regression line is sytematically over- and under predicting the data. We can check this by plotting the residuals, these are defined by the difference of the observed value and the predicted value. Below the residuals are plotted.

![Residual plot](plots/residual_plot.png)

For a good model fit, the residuals shouldn't be very large or very small all of the time, because the residual is in fact a stochastic error and we should not be able to predict it. Therefore the residuals should be centered around zero and follow a normal distribution. The residual plot looks quite like a normal distribution, so we can say that the model is good.

## Visualization of ridership by time of day

Below a visualization can be found about the ridership by time of day. On the x-axis the hour is give and on the y-axis the average number of entries. The red color indicates no rain and the blue color indicates rain situation. At the start of the day there is almost no difference, but when the day progresses and it gets busier, the difference get bigger (except for the dip at 16h).

![Histogram of rain vs norain values](plots/avg_number_of_entries_per_hour.png)


## Conclusion

#### Do more people ride the NYC subway when it is raining or when it is not raining?

From the results obtained from the statistical test and lineair regression, we can say in general that more people ride the NYC subway when it is raining. However, rain is not a very good predictor.

In the statistical test, the conclusion is that our sample supports the alternative hypothesis, the population mean for rain is greater that for non rain.

In our lineair regression model, the influence of the rain on the ouput is negative. I have also tried to leave out the feature rain and it almost didn't change the R2 value. The purpose of the regression model is to be able to predict the outcome based on certain features. However, these features influence each other. If one is left out, it will change the param values of the other features. The dummy variable UNIT has a big influence on R2. This leads to the conclusion that UNIT is a good predictor, but rain isn't.


## Reflection

#### Shortcomings

There are some shortcomings in my analysis.
* dataset: the dataset contains only data from 1 month (May 2011), this is quite a short period. It might be a month with a lot of rain that is not representable or it could be a time when there are a lot of people on vacation.
The dataset also contains some columns that seem to be closely related to each other. For example, the daily average of the pressure (meanpressurei) and the  pressure at the time/location itself (pressurei), this also counts for temperature. When using a lineair regression model with these columns as predictors, it might not give valid results for individual predictors, this problem is called multicollinearity.
* regression model: the chosen regression model OLS (ordinary least squares) is quite simplistic and has some drawbacks. It is sensitive for outliers and as seen in the figure of entries per hour, there are some outliers. Next to this, OLS performs bad when features are put into the model, that have a dependence between them. 

